In [70]:
import os

import pandas as pd
import numpy as np

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from bayes_opt import BayesianOptimization
import lightgbm as lgb

from sklearn.feature_selection import mutual_info_regression, SelectKBest, f_regression
from sklearn.utils import resample
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


from sklearn.metrics import confusion_matrix
random_state=42

In [2]:
df= pd.read_csv('/Users/josephlim/Desktop/Data Science/Capstone Projects/Capstone project- Spotify/Data/Cleaned Data/spotify_data_preprocessed.csv')

In [3]:
df.head()

,popularity,duration_ms,danceability,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0.14,266773,0.429,-7.227,0.0281,0.00239,0.000121,0.2340,0.285,173.372,4
1,0.33,201960,0.659,-5.850,0.0437,0.00450,0.000002,0.3350,0.798,106.965,4
2,0.16,216880,0.556,-5.870,0.0584,0.00958,0.000000,0.2090,0.400,105.143,4
3,0.16,284200,0.949,-4.244,0.0572,0.03020,0.000000,0.0454,0.760,104.504,4
4,0.63,201960,0.659,-5.850,0.0437,0.00450,0.000002,0.3350,0.798,106.965,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92194 entries, 0 to 92193
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   popularity        92194 non-null  float64
 1   duration_ms       92194 non-null  int64  
 2   danceability      92194 non-null  float64
 3   loudness          92194 non-null  float64
 4   speechiness       92194 non-null  float64
 5   acousticness      92194 non-null  float64
 6   instrumentalness  92194 non-null  float64
 7   liveness          92194 non-null  float64
 8   valence           92194 non-null  float64
 9   tempo             92194 non-null  float64
 10  time_signature    92194 non-null  int64  
dtypes: float64(9), int64(2)
memory usage: 7.7 MB


In [5]:
X= df.drop('popularity', axis=1)
y= df['popularity']

Let's split the data into training and testing sets.

In [6]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.3, random_state=42)

In [7]:
df.describe()

,popularity,duration_ms,danceability,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,92194.000000,9.219400e+04,92194.000000,92194.000000,92194.00000,92194.000000,92194.000000,92194.000000,92194.000000,92194.000000,92194.000000
mean,0.620856,2.385335e+05,0.600292,-7.563552,0.08614,0.306901,0.063995,0.207626,0.545761,121.392948,3.925559
std,0.162090,1.132078e+05,0.159702,3.763095,0.11218,0.295936,0.205188,0.184344,0.252382,29.337843,0.364847
min,0.060000,6.360000e+03,0.000000,-60.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.510000,1.949470e+05,0.498000,-9.042000,0.03320,0.040300,0.000000,0.096000,0.343000,97.951500,4.000000
50%,0.610000,2.288025e+05,0.611000,-6.816000,0.04480,0.208000,0.000003,0.132000,0.549000,120.379000,4.000000
75%,0.720000,2.687652e+05,0.716000,-5.143000,0.08200,0.530000,0.000607,0.266000,0.758000,139.847750,4.000000
max,1.000000,4.995083e+06,0.991000,2.854000,0.96200,0.996000,1.000000,1.000000,1.000000,220.230000,5.000000


### Dummy Regressor

Let's explore different models we can predict popularity by. A good starting point would be to see how good the mean would be as a predictor. DummyRegressor can do just that.

In [8]:
train_mean= y_train.mean()
train_mean

0.6206697141086231

In [9]:
dumb_reg= DummyRegressor(strategy='mean')

dumb_reg.fit(X_train, y_train)
dumb_reg.score(X_test, y_test)

-1.4600769697281635e-05

It comes pretty close for now, but how good is this? We'll see how closely this explains the actual values. There are many metrics we can use to do this. We will try different metrics to choose which one works best for our specific scenario.

## Metrics

**1. Coefficient of determination ($R^{2}$):**
<br> We will make predictions by creating length of size of training set with the single value of the mean.

In [10]:
y_tr_pred_ = train_mean* np.ones(len(y_train))
y_tr_pred_[:5]

array([0.62066971, 0.62066971, 0.62066971, 0.62066971, 0.62066971])

In [11]:
y_tr_pred= dumb_reg.predict(X_train)
y_tr_pred[:5]

array([0.62066971, 0.62066971, 0.62066971, 0.62066971, 0.62066971])

They produce exactly same results. Let's find out ($R^{2}$) score.

In [12]:
r2_score(y_train, y_tr_pred)

0.0

We got $R^{2}$ score of 0 on our training set when we used mean as a predictor. $R^{2}$ explains predictions in terms of the amount of variance. Low $R^{2}$ score suggests small amount of variances explainedn. Let's try this on our test set. 

In [13]:
y_te_pred= train_mean*np.ones(len(y_test))
r2_score(y_test, y_te_pred)

-1.4600769697281635e-05

This negative number was expected, because most models perform worse on test sets than on training sets.

**2.Mean Absolute Error (MAE)**
<br>
MAE tells us how much we expect to be off by if we guessed based on the average of known values.

In [14]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

(0.1278059219662443, 0.12801506385334058)

MAE on testing set is very slightly worse than on training data, but they are very similar.

**3.Mean Squared Error (MSE)**
<br>Mean squared error is the average of square of errors.

In [15]:
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

(0.026265295652843034, 0.026290308453708538)

While it was pretty obvious from $R^{2}$ score that a simple average would not be the best model, it was worth double (or triple) checking it using different metrics. 

## Initial Models:

Let's build pipelines to simplify processes. 

In [16]:
pipe = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

In [17]:
type(pipe)

sklearn.pipeline.Pipeline

In [18]:
hasattr(pipe, 'fit'), hasattr(pipe, 'predict')

(True, True)

Let's try fitting, making predictions, and evaluating performance using this pipeline.

In [19]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [20]:
y_tr_pred= pipe.predict(X_train)
y_te_pred= pipe.predict(X_test)

In [21]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.05735959711767247, 0.05996889136075756)

Given the number of features, it is possible that I am overfitting. As such, I will limit number of features that get used based on f_regression.

In [22]:
pipe = make_pipeline(
    StandardScaler(),
    SelectKBest(f_regression),
    LinearRegression()
)

In [23]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('selectkbest',
                 SelectKBest(score_func=<function f_regression at 0x7f83c3492790>)),
                ('linearregression', LinearRegression())])

In [24]:
y_tr_pred = pipe.predict(X_train)
y_te_pred = pipe.predict(X_test)

In [25]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.05735959711767247, 0.05996889136075756)

In [26]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

(0.12512771154799862, 0.12530144115509462)

We see that our model performance got much worse than when we didn't limit features. This was an expected result of limiting feature numbers. Let's try to include more.

In [27]:
pipe10 = make_pipeline(
    StandardScaler(),
    SelectKBest(f_regression, k=10),
    LinearRegression()
)

In [28]:
y_tr_pred = pipe.predict(X_train)
y_te_pred = pipe.predict(X_test)

In [29]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.05735959711767247, 0.05996889136075756)

In [30]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

(0.12512771154799862, 0.12530144115509462)

It didn't make too much of a difference. Let's try to use cross-validation to assess model performance.

In [31]:
cv_results= cross_validate(pipe10, X_train, y_train, cv=5)

In [32]:
cv_scores= cv_results['test_score']
cv_scores

array([0.06189826, 0.05326071, 0.05647042, 0.05476803, 0.05775996])

In [34]:
np.mean(cv_scores), np.std(cv_scores)

(0.05683147698481341, 0.0029554351736718943)

We can estimate the variability, or uncertainty of model performance.

In [35]:
np.round((np.mean(cv_scores) - 2 * np.std(cv_scores), np.mean(cv_scores) + 2 * np.std(cv_scores)), 2)

array([0.05, 0.06])

## Linear Regression Model

### GridSearchCV - Linear Regression

In [ ]:
pipe.get_params().keys()

In [ ]:
k = [k+1 for k in range(len(X_train.columns))]
grid_params = {'selectkbest__k': k}

In [ ]:
lr_grid_cv = GridSearchCV(pipe, param_grid=grid_params, cv=5, n_jobs=-1)

In [ ]:
lr_grid_cv.fit(X_train, y_train)

In [ ]:
score_mean = lr_grid_cv.cv_results_['mean_test_score']
score_std = lr_grid_cv.cv_results_['std_test_score']
cv_k = [k for k in lr_grid_cv.cv_results_['param_selectkbest__k']]


In [ ]:
lr_grid_cv.best_params_

In [ ]:
best_k = lr_grid_cv.best_params_['selectkbest__k']
plt.subplots(figsize=(10, 5))
plt.errorbar(cv_k, score_mean, yerr=score_std)
plt.axvline(x=best_k, c='r', ls='--', alpha=.5)
plt.xlabel('k')
plt.ylabel('CV score (r-squared)')
plt.title('Pipeline mean CV score (error bars +/- 1sd)')

In [ ]:
selected = lr_grid_cv.best_estimator_.named_steps.selectkbest.get_support()

In [ ]:
coefs= lr_grid_cv.best_estimator_.named_steps.linearregression.coef_
features= X_train.columns[selected]
pd.Series(coefs, index= features).sort_values(ascending= False)

We see that the most positive feature were instrumentalness. This differs from our EDA, which showed preference to non-instrumentalness. Let's try a different model.

## Random Forest Model

In [ ]:
RF= RandomForestRegressor(random_state= 42)

In [ ]:
rf_default_cv_results= cross_validate(RF, X_train, y_train, cv=5)

In [ ]:
rf_cv_scores= rf_default_cv_results['test_score']
rf_cv_scores

In [ ]:
np.mean(rf_cv_scores), np.std(rf_cv_scores)

Random Forest Regression model looks much more promising. Let's see what hyperparameter tuning can do for this model.

#### Random Forest Model- Bayesian Optimization

In [ ]:
cv_results.keys()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
def fit_model(n_estimators, min_samples_split, max_depth, max_features):
    num_leaves= round(num_leaves)
    max_depth= round(max_depth)
    min_child_samples= round(min_child_samples)
    min_data_in_leaf= round(min_data_in_leaf)
    
    regressor= RandomForestRegressor(num_leaves= num_leaves, 
                                 max_depth=max_depth,
                                 min_child_samples= min_child_samples, 
                                 random_state=random_state)

    return np.mean(cross_validate(regressor, X_train, y_train, scoring='r2', cv=5, return_train_score=True)['test_score'])


In [ ]:
rf_BO = BayesianOptimization(fit_model,{
        "n_estimators":(300, 2000), 
        "min_samples_split":(20,100), 
        "max_depth":(3,7), 
        "max_features":(5,13) 
    })

rf_BO.maximize(n_iter=10, init_points=2)

### Gradient Boosting Model
The performance of Random Forest Model wasn't as good as I hoped. Let's try a gradient boosting model.

#### Gradient Boosting Model- Bayesian Optimization

In [36]:
def lgb_eval(num_leaves, max_depth, min_child_samples, min_data_in_leaf):
    num_leaves= round(num_leaves)
    max_depth= round(max_depth)
    min_child_samples= round(min_child_samples)
    min_data_in_leaf= round(min_data_in_leaf)
    
    regressor= lgb.LGBMRegressor(num_leaves= num_leaves, 
                                 max_depth=max_depth,
                                 min_child_samples= min_child_samples, 
                                 random_state=42)
    
    return np.mean(cross_validate(regressor, X_train, y_train, scoring='r2', cv=5, error_score= 'raise')['test_score'])

In [37]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (25, 4000),
                                                'max_depth': (5, 63),
                                                'min_child_samples': (50, 10000),
                                                'min_data_in_leaf': (100, 2000)
                                                })


lgbBO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | max_depth | min_ch... | min_da... | num_le... |
-------------------------------------------------------------------------
| 1         | 0.1199    | 22.21     | 2.954e+03 | 885.3     | 3.693e+03 |
| 2         | 0.1462    | 46.89     | 297.6     | 827.5     | 596.8     |
| 3         | 0.09064   | 8.214     | 8.122e+03 | 572.3     | 2.456e+03 |
| 4         | 0.1238    | 14.84     | 2.496e+03 | 1.961e+03 | 45.2      |
| 5         | 0.143     | 58.84     | 763.7     | 1.149e+03 | 1.698e+03 |
| 6         | 0.1339    | 43.54     | 53.57     | 373.5     | 3.718e+03 |
| 7         | 0.1368    | 59.02     | 74.75     | 1.984e+03 | 1.082e+03 |
| 8         | 0.1419    | 32.54     | 72.63     | 178.7     | 1.063e+03 |
| 9         | 0.1331    | 23.31     | 1.597e+03 | 142.3     | 96.56     |
| 10        | 0.1124    | 48.83     | 4.139e+03 | 1.574e+03 | 1.946e+03 |
| 11        | 0.1286    | 9.406     | 1.953e+03 | 1.531e+03 | 1.105e+03 |
| 12        | 0.1399    | 53.35     | 

In [38]:
print(lgbBO.max)

{'target': 0.14623287399928964, 'params': {'max_depth': 46.89209917857136, 'min_child_samples': 297.64712942008214, 'min_data_in_leaf': 827.5177278231067, 'num_leaves': 596.8281699655965}}


Our prediction actually worsened. For now, our best bet is Random Forest regression model. However, I'm still not too excited about our current model performance. Perhaps we should try our hands at classification models.

### Classifications
We'll try to predict songs' popularities by categorizing popularities(into "high","mid","low"), and classifying songs into those categories.

In [41]:
pd.cut(df['popularity'], bins=3)

0        (0.0591, 0.373]
1        (0.0591, 0.373]
2        (0.0591, 0.373]
3        (0.0591, 0.373]
4         (0.373, 0.687]
              ...       
92189    (0.0591, 0.373]
92190    (0.0591, 0.373]
92191     (0.373, 0.687]
92192    (0.0591, 0.373]
92193     (0.373, 0.687]
Name: popularity, Length: 92194, dtype: category
Categories (3, interval[float64, right]): [(0.0591, 0.373] < (0.373, 0.687] < (0.687, 1.0]]

In [42]:
labels=['low','medium', 'high']
df['popularity']= pd.cut(df['popularity'], bins=3, labels=labels, right=True)

In [43]:
df.popularity.value_counts()

medium    57662
high      29518
low        5014
Name: popularity, dtype: int64

The dataset is unbalanced. This is intuitive, because there aren't as many popular songs as there are non-popular songs (otherwise, there will be much more financial stability in music industry!). However,imbalance in dataset will tamper with the accuracy of our model. One way to counteract this is by upsampling songs with high popularity.  We will then perform K-Nearest Neighbor classification, because they are good at handling noisy data. 

#### Up-sampling songs with "high" and "low" popularity

In [44]:
df_high= df[df.popularity=='high']
df_mid= df[df.popularity=='medium']
df_low= df[df.popularity=='low']

df_low_upsampled= resample(df_low, replace=True, n_samples= 57662, random_state=42)
df_high_upsampled= resample(df_high, replace=True, n_samples=57662, random_state=42)


list_df_upsampled=[df_high_upsampled, df_mid, df_low_upsampled]
df_upsampled= pd.concat(list_df_upsampled)

In [45]:
df_upsampled.popularity.value_counts()

low       57662
medium    57662
high      57662
Name: popularity, dtype: int64

In [46]:
X_up= df_upsampled.drop('popularity', axis=1)
y_up= df_upsampled['popularity']

In [47]:
X_train, X_test, y_train, y_test= train_test_split(X_up, y_up, random_state= 42, test_size=0.3)

#### Testing Different Classification Models

**KNN Classifier** 

In [ ]:
KNN= KNeighborsClassifier()

KNN.fit(X_train, y_train)

y_pred_classification= KNN.predict(X_test)
print(accuracy_score(y_test, y_pred_classification))

WOW! This is a huge improvement (though it probably is overfitting). Let's try Random Forest Classification.

**Random Forest Classifier**

In [ ]:
RFC= RandomForestClassifier(random_state= random_state)

RFC.fit(X_train, y_train)
y_rfc_pred= RFC.predict(X_test)
print(accuracy_score(y_test, y_rfc_pred))

Clearly, Random Forest Classifier performs best for our case. Let's use this model. Before fully delving into this model though, let's check for feature importance to see which features are more relevant for our task.

**Feature Importances**

In [ ]:
importances= pd.Series(data= RFC.feature_importances_, index=X_train.columns)
importances_sorted= importances.sort_values()

importances_sorted.plot(kind='barh', color='blue')
plt.title('Feature Importances')
plt.show()

We see that loudness plays and duration plays the biggest role in predicting popularity. We then see that energy, valence, speechiness, acousticness, danceability, tempo, and liveness, in that order, influence our predictions.

#### Random Forest Classification- Bayesian Optimization

In [78]:
def fit_model(n_estimators, min_samples_split, max_depth, max_leaf_nodes):
    n_estimators= round(n_estimators)
    min_samples_split= round(min_samples_split) 
    max_depth= round(max_depth)
    max_leaf_nodes= round(max_leaf_nodes)
   
    regressor= RandomForestClassifier(n_estimators= n_estimators, 
                                 min_samples_split= min_samples_split,
                                 max_depth=max_depth,
                                 random_state=random_state)

    return np.mean(cross_validate(regressor, X_train, y_train, scoring='accuracy', error_score= 'raise', cv=5)['test_score'])

In [ ]:
rf_BO = BayesianOptimization(fit_model,{
        'n_estimators': (5,2000),
        'min_samples_split':(1,100), 
        'max_depth': (1,10),
        'max_leaf_nodes': (2,10)
    })

rf_BO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | max_depth | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.4526    | 2.822     | 9.692     | 99.47     | 1.553e+03 |


In [ ]:
print(rf_BO.max)

In [ ]:
Maybe try RandomizedSearchCV : https://www.geeksforgeeks.org/random-forest-hyperparameter-tuning-in-python/

In [71]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

        high       0.83      0.86      0.84     16613
         low       1.00      0.98      0.99     17795
      medium       0.85      0.83      0.84     17488

    accuracy                           0.89     51896
   macro avg       0.89      0.89      0.89     51896
weighted avg       0.89      0.89      0.89     51896



In [75]:
param_grid = {
    'n_estimators': [25, 50, 100, 150, 1000, 2000],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

In [76]:
random_search = RandomizedSearchCV(RandomForestClassifier(),
                                   param_grid)
random_search.fit(X_train, y_train)
print(random_search.best_estimator_)

RandomForestClassifier(max_depth=6, max_features='sqrt', max_leaf_nodes=9,
                       n_estimators=150)


In [77]:
final_model= RandomForestClassifier(max_depth=6, max_features='sqrt', max_leaf_nodes=9,
                       n_estimators=150)

final_model.fit(X_train, y_train)
y_pred_final= final_model.predict(X_test)
print(classification_report(y_pred_final, y_test))

              precision    recall  f1-score   support

        high       0.58      0.46      0.51     21941
         low       0.61      0.49      0.54     22036
      medium       0.17      0.38      0.24      7919

    accuracy                           0.46     51896
   macro avg       0.46      0.44      0.43     51896
weighted avg       0.53      0.46      0.48     51896



Through hyperparameter tuning, we learned that the model can expect target score of 0.86. This score is reasonable, as the model is expected to perform better on training set. However, our current target score is still great. Let's use confusion matrix to further evaluate our model.

### Confusion Matrix

In [ ]:
cmat = confusion_matrix(y_test, y_rfc_pred)
#print(cmat)
print('TP - True Negative {}'.format(cmat[0,0]))
print('FP - Flase Positive {}'.format(cmat[0,1]))
print('FN - False Negative {}'.format(cmat[1,0]))
print('TP - True Positive {}'.format(cmat[1,1]))
print('Accuracy Score: {}'.format(np.divide(np.sum([cmat[0,0], cmat[1,1], cmat[2,2]]), np.sum(cmat)))) 
print('Misclassification Rate: {}'.format(np.divide(np.sum([cmat[1,0], cmat[0,1], cmat[0,2], cmat[2,0], cmat[1,2], cmat[2,1]]), np.sum(cmat))))

An accuracy score of 89% is great! We came a long way from our linear regression model. 